In [3]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')

In [34]:
vod = pd.read_csv('../data/vod89.csv')
vod

,userid,program,score,main_cat,sub_cat,category,rename
0,59879000,소방서 옆 경찰서,0.244476,TV드라마,기타,TV프로그램,소방서옆경찰서
1,59879000,신성한 이혼,0.738281,TV드라마,기타,TV프로그램,신성한이혼
2,59895000,금이야 옥이야,1.000000,TV드라마,기타,TV프로그램,금이야옥이야
3,59900000,2022 역사저널 그날,0.215482,TV 시사/교양,기타,TV프로그램,2022역사저널그날
4,59900000,후궁 제왕의첩,0.292893,영화,멜로,영화,후궁제왕의첩
...,...,...,...,...,...,...,...
1498,67140000,밀수,0.250000,영화,액션/어드벤쳐,영화,밀수
1499,67140000,경남 통영 2부,0.292893,우리동네,연예/오락,TV프로그램,경남통영2부
1500,67140000,잠자는 숲속의 공주,0.292893,키즈,기타,키즈,잠자는숲속의공주
1501,67148000,타요의 씽씽극장 동요2,0.988951,키즈,기타,키즈,타요의씽씽극장동요2


In [36]:
vod_id = pd.read_csv('../data/vod_add_id.csv', index_col=0)
vod_id = vod_id[['rename', 'Category', 'vod_id']]
vod_id.columns = ['program', 'vod_id']
vod_id

,rename,Category,vod_id
0,와이낫크루뜻밖의여행,TV프로그램,0
1,그레이트뷰티,영화,1
2,해안선,영화,2
3,아는형님,TV프로그램,3
4,베놈,영화,4
...,...,...,...
4710,베놈2:렛데어비카니지,영화,4710
4711,성월동화,영화,4711
4712,다큐인사이트코드블루,TV프로그램,4712
4713,캠프제로베이스원,TV프로그램,4713


In [32]:
vod1 = vod.merge(vod_id, on = ['program'], how = 'left')
vod1

,userid,program,score,main_cat,sub_cat,category,vod_id
0,59879000,소방서 옆 경찰서,0.244476,TV드라마,기타,TV프로그램,NaN
1,59879000,신성한 이혼,0.738281,TV드라마,기타,TV프로그램,NaN
2,59895000,금이야 옥이야,1.000000,TV드라마,기타,TV프로그램,NaN
3,59900000,초대: 스와핑 데이,0.292893,영화,멜로,영화,NaN
4,59900000,후궁 제왕의첩,0.292893,영화,멜로,영화,NaN
...,...,...,...,...,...,...,...
1503,67140000,밀수,0.250000,영화,액션/어드벤쳐,영화,2865.0
1504,67140000,잠자는 숲속의 공주,0.292893,키즈,기타,키즈,NaN
1505,67140000,경남 통영 2부,0.292893,우리동네,연예/오락,TV프로그램,NaN
1506,67148000,타요의 씽씽극장 동요2,0.988951,키즈,기타,키즈,NaN


In [7]:
from sklearn.utils import shuffle
TRAIN_SIZE = 0.75
vod = shuffle(vod)
cutoff = int(TRAIN_SIZE * len(vod))

vod_train = vod.iloc[:cutoff]
vod_test = vod.iloc[cutoff:]

In [8]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dot, Add, Flatten
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD, Adamax

# variable 초기화
K = 200
mu = vod_train.score.mean()
M = vod.userid.max

,userid,program,score,main_cat,sub_cat,category
272,61113000,TV동물농장,0.250000,TV 연예/오락,기타,TV프로그램
699,64364000,인어공주,0.292893,영화,SF/환타지,영화
345,61767000,미녀 특공대,0.292893,영화,액션/어드벤쳐,영화
1126,66295000,런민기TV 시즌4 카러플,0.292893,TV애니메이션,기타,TV프로그램
284,61191000,아씨두리안,0.875000,TV드라마,기타,TV프로그램
...,...,...,...,...,...,...
1254,66705000,일급 살수,0.292893,영화,액션/어드벤쳐,영화
729,64385000,내일은 위닝샷,0.562500,TV 연예/오락,기타,TV프로그램
584,63411000,TV동물농장,0.300000,TV 연예/오락,기타,TV프로그램
497,62701000,궁금한이야기Y,0.397790,TV 연예/오락,기타,TV프로그램
